In [1]:
pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=1e9a6709c618695745b6419ee2dd173405f2e551d3a4f5e978d8d3d1f28d1b9b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [9]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('3').getOrCreate()

from pyspark.sql.functions import *
from pyspark.sql.types import *

In [10]:
data = spark.read.csv('/content/weather_data.csv', header = True, inferSchema = True)

In [11]:
data.show()

+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|_c0|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|
+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|  0|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|
|  1|1902| 19.0|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|
|  2|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|
|  3|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|
|  4|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71|
|  5|1906| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.82|26.23|24.75|21.93|19.55|
|  6|1907|19.27|19.42|22.03|26.98|27.52|27.66|27.28|26.38|26.26|24.72|22.11|18.46|
|  7|1908|18.35|19.73|22.93|27.06|28.07|28.49|27.16|26.41|25.74|24.25|21.06|18.15|
|  8|1909|17.79|19.05| 23.4|25.76|27.97|27.67|26.56|26.43|25.47|24.37|22.01|18.86|
|  9

####  Add column average temperature for each year

In [12]:
columns = ['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC']
expressions = '+'.join(columns)
new_data = data.withColumn('Total',expr(expressions))
new_data = new_data.withColumn('Average', col('Total')/12)
new_data.show()

+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+------------------+
|_c0|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|             Total|           Average|
+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+------------------+
|  0|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|            290.69|24.224166666666665|
|  1|1902| 19.0|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|291.91999999999996|24.326666666666664|
|  2|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|            287.77|23.980833333333333|
|  3|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|             286.2|23.849999999999998|
|  4|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71|            287.19|           23.9325|
|  5|1906| 17.5|19.14|22.21|26.5

#### 2. Add row average temperature of each month

In [13]:
monthly_avg = new_data.groupBy().agg(*[avg(col) for col in new_data])
new_data_2 = new_data.union(monthly_avg)
monthly_avg.show()

+--------+---------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+
|avg(_c0)|avg(YEAR)|         avg(JAN)|          avg(FEB)|          avg(MAR)|          avg(APR)|          avg(MAY)|          avg(JUN)|          avg(JUL)|          avg(AUG)|          avg(SEP)|          avg(OCT)|          avg(NOV)|          avg(DEC)|       avg(Total)|      avg(Average)|
+--------+---------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+
|    58.0|   1959.0|18.42324786324786|20.137948717948717|23.434188034188036|26.514102564102565|28.386410256410255|28.300940170940162|27.369230769

#### 3. Add columns Min and Max for each year with corresponding values

In [15]:
new_data3 = new_data.withColumn('min', least('JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'))
new_data3 = new_data3.withColumn('max', greatest('JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'))
new_data3.show()

+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+------------------+-----+-----+
|_c0|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|             Total|           Average|  min|  max|
+---+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------------------+------------------+-----+-----+
|  0|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|            290.69|24.224166666666665|17.99| 28.6|
|  1|1902| 19.0|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|291.91999999999996|24.326666666666664|18.78|28.68|
|  2|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|            287.77|23.980833333333333|18.29|28.41|
|  3|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|             286.2|23.849999999999998|17.77|27.85|
|  4|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16

####  Create dataframe called 'decade', storing average temperatures of each month in the respective decade. Decade 1910 is from 1901 to 1910.

In [16]:
new_data4 = new_data3.select('YEAR','JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC').filter(col('YEAR')>1900).filter(col('YEAR')<1911)
new_data4.show()

+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|YEAR|  JAN|  FEB|  MAR|  APR|  MAY|  JUN|  JUL|  AUG|  SEP|  OCT|  NOV|  DEC|
+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|1901|17.99|19.43|23.49|26.41|28.28| 28.6|27.49|26.98|26.26|25.08|21.73|18.95|
|1902| 19.0|20.39| 24.1|26.54|28.68|28.44|27.29|27.05|25.95|24.37|21.33|18.78|
|1903|18.32|19.79|22.46|26.03|27.93|28.41|28.04|26.63|26.34|24.57|20.96|18.29|
|1904|17.77|19.39|22.95|26.73|27.83|27.85|26.84|26.73|25.84|24.36|21.07|18.84|
|1905| 17.4|17.79|21.78|24.84|28.32|28.69|27.67|27.47|26.29|26.16|22.07|18.71|
|1906| 17.5|19.14|22.21|26.53|29.06|28.02|27.46|26.82|26.23|24.75|21.93|19.55|
|1907|19.27|19.42|22.03|26.98|27.52|27.66|27.28|26.38|26.26|24.72|22.11|18.46|
|1908|18.35|19.73|22.93|27.06|28.07|28.49|27.16|26.41|25.74|24.25|21.06|18.15|
|1909|17.79|19.05| 23.4|25.76|27.97|27.67|26.56|26.43|25.47|24.37|22.01|18.86|
|1910|18.14|19.72| 22.9|25.96|28.36|27.72|26.93|26.6

In [17]:
new_data4 = new_data4.drop('YEAR')
monthly_avg_decade = new_data4.agg(*[avg(col) for col in new_data4])
decade = new_data4.union(monthly_avg_decade)

#### 5. Which year is the hottest year? (Based on avg temp)

In [18]:
new_data3.select('YEAR','Average').orderBy('Average', ascending = False).collect()[1]

Row(YEAR=2017, Average=26.291666666666668)

#### 6. Which year is the coldest year? (Based on avg temp)


In [19]:
new_data3.select('YEAR','Average').orderBy('Average').collect()[1]

Row(YEAR=1950, Average=23.735)

#### 7. Find out which years recorded minimum and maximum temperatures. (Not based on avg temp)


In [20]:
new_data3.select('YEAR','min').orderBy('min').collect()[1]
new_data3.select('YEAR','max').orderBy('max', ascending = False).collect()[1]

Row(YEAR=2017, max=30.47)

#### 8. How much raise is observed in min and max temperature for each month?

In [21]:
new_data6 = new_data3.select('JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC')
monthly_max = new_data6.agg(*[max(col) for col in new_data6])
monthly_min = new_data6.agg(*[min(col) for col in new_data6])
monthly_max.show()
monthly_min.show()

+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|max(JAN)|max(FEB)|max(MAR)|max(APR)|max(MAY)|max(JUN)|max(JUL)|max(AUG)|max(SEP)|max(OCT)|max(NOV)|max(DEC)|
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|   20.92|   23.58|   26.61|   29.56|   30.78|   29.88|   28.47|   28.17|   28.11|   27.24|   23.92|   21.89|
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+

+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|min(JAN)|min(FEB)|min(MAR)|min(APR)|min(MAY)|min(JUN)|min(JUL)|min(AUG)|min(SEP)|min(OCT)|min(NOV)|min(DEC)|
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|   17.25|   17.79|   21.78|   24.84|   26.97|   27.33|   26.48|   26.21|   25.47|   23.52|   20.59|   17.98|
+--------